# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv from WeatherPy city_weather_df
city_weather_df = pd.read_csv("../output_data/cities.csv")
city_weather_df


,City,Lat,lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bethel,41.3712,-73.4140,89.35,61,40,1.99,US,1626465896
1,Lufilufi,-13.8745,-171.5986,81.32,78,33,23.55,WS,1626465727
2,Khanbari,27.3747,87.2039,73.31,94,100,0.87,NP,1626465896
3,Quime,-16.9833,-67.2167,61.84,43,3,5.95,BO,1626465897
4,Cape Town,-33.9258,18.4232,49.95,95,0,4.76,ZA,1626465620
...,...,...,...,...,...,...,...,...,...
557,Msanga,-6.0500,36.0333,65.86,49,39,6.60,TZ,1626466127
558,Verkhoyansk,67.5447,133.3850,63.03,47,100,3.22,RU,1626466127
559,La Paz,-16.5000,-68.1500,57.51,29,1,10.36,BO,1626465865
560,Wadi Maliz,36.4681,8.5495,70.43,73,0,4.90,TN,1626466128


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = city_weather_df[["Lat", "lng"]].astype(float)
humidity = city_weather_df["Humidity"].astype(float)

In [4]:
# Add Heatmap layer to the map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Look at the data by adding filters per the instructions
(city_weather_df['Temp']<=80) & (city_weather_df['Temp']>=70) & (city_weather_df['Wind Speed']<=10) & (city_weather_df['Cloudiness'] == 0)


0      False
1      False
2      False
3      False
4      False
       ...  
557    False
558    False
559    False
560     True
561    False
Length: 562, dtype: bool

In [6]:
# Update the DataFrame with added filters
update_df = city_weather_df.loc[(city_weather_df['Temp']<=80) & (city_weather_df['Temp']>=70) & (city_weather_df['Wind Speed']<=10) & (city_weather_df['Cloudiness'] == 0)
]
update_df

,City,Lat,lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,São Filipe,14.8961,-24.4956,74.73,79,0,7.25,CV,1626465899
22,Thessalon,46.2501,-83.5666,77.31,47,0,4.79,CA,1626465904
104,Muros,42.7762,-9.0603,79.90,51,0,4.94,ES,1626465938
109,Severnyy,50.6770,36.5532,73.44,78,0,6.71,RU,1626465940
209,Tazmalt,36.3868,4.3989,74.75,52,0,5.57,DZ,1626465983
218,Albania,41.0000,20.0000,70.99,50,0,1.63,AL,1626465986
250,Karla,59.3723,24.9275,75.09,65,0,4.61,EE,1626465999
256,Felanitx,39.4696,3.1483,74.93,70,0,3.20,ES,1626466002
269,Zhangye,38.9342,100.4517,76.86,35,0,4.52,CN,1626466007
283,Caravelas,-17.7125,-39.2481,74.14,80,0,8.43,BR,1626466013


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = update_df[["City", "Country", "Lat", "lng"]]

hotel_df["Hotel Name"] = ""

hotel_df

C:\Users\pugha\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,Lat,lng,Hotel Name
9,São Filipe,CV,14.8961,-24.4956,
22,Thessalon,CA,46.2501,-83.5666,
104,Muros,ES,42.7762,-9.0603,
109,Severnyy,RU,50.6770,36.5532,
209,Tazmalt,DZ,36.3868,4.3989,
218,Albania,AL,41.0000,20.0000,
250,Karla,EE,59.3723,24.9275,
256,Felanitx,ES,39.4696,3.1483,
269,Zhangye,CN,38.9342,100.4517,
283,Caravelas,BR,-17.7125,-39.2481,


In [8]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotels",
    "key": g_key}

In [9]:
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["lng"]

    # update address key value
    params['location'] = f"{lat},{lng}"

    # make request
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

C:\Users\pugha\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.


In [10]:
hotel_df

,City,Country,Lat,lng,Hotel Name
9,São Filipe,CV,14.8961,-24.4956,La Fora Ecolodge
22,Thessalon,CA,46.2501,-83.5666,Carolyn Beach Inn & Restaurant
104,Muros,ES,42.7762,-9.0603,Hotel do Porto
109,Severnyy,RU,50.6770,36.5532,Hotel VOYAGE
209,Tazmalt,DZ,36.3868,4.3989,Hotel Rabah Syphax
218,Albania,AL,41.0000,20.0000,Bora Homestay
250,Karla,EE,59.3723,24.9275,
256,Felanitx,ES,39.4696,3.1483,HOTEL RURAL SA POSADA D'AUMALLIA
269,Zhangye,CN,38.9342,100.4517,Holiday Inn Express Zhangye
283,Caravelas,BR,-17.7125,-39.2481,Mar Doce de Abrolhos


In [12]:
# NOTE: Do not change any of the code in this cell
## Note that I had to change Lng to lng as I didn't capialize it above ##

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))